In [1]:
#需要先修改excel，比如 excel里面龚婕(龚洁)，而EEG数据为AV-龚洁-20220627，此时请将excel中的龚婕(龚洁)改为龚洁
#否则排序和匹配会出问题
#还要去掉AV-周牧涵-20220518 这个有误的数据
#将第二批刺激参数表的龙元芳 20220516的标签改为-，因为并没有这个数据

In [2]:
from mne.io import concatenate_raws, read_raw_edf
import mne
import glob
import numpy as np
import xlrd
import pandas as pd

In [3]:
#category为train，就会进行数据增强；为test则不进行数据增强
category="test"

In [4]:
parameter_path=['../SE_parameter/刺激参数（第一批次）.xlsx']

In [5]:
# 性别和年龄暂时不用，且第二批数据没有这两项
df=pd.DataFrame(columns=['姓名','治疗时间','刺激方案','标签'])
df

,姓名,治疗时间,刺激方案,标签


In [6]:
for path in parameter_path:
    df = pd.concat([df,pd.read_excel(path,engine='openpyxl')],ignore_index=True,join='inner')
df

,姓名,治疗时间,刺激方案,标签
0,曾刚,20220216,2,1
1,曾刚,20220218,3,0
2,曾刚,20220221,4,1
3,曾刚,20220223,1,1
4,杜娇,20220316,4,1
...,...,...,...,...
91,赵丽兰,20220304,3,1
92,周小凯,20220314,2,0
93,周小凯,20220316,4,1
94,周小凯,20220318,1,1


In [7]:
# path = "../SE_parameter/刺激参数（第四批次）.xlsx"
# df = pd.read_excel(path,engine='openpyxl')
# #df= df.astype({'完成时间':'object'})
# #对df进行排序，为了和glob.glob的顺序相匹配
# df

In [8]:
df=df.sort_values(by=["姓名","治疗时间"])
df

,姓名,治疗时间,刺激方案,标签
8,何富桂,20220221,1,1
9,何富桂,20220225,4,1
10,何富桂,20220228,3,1
11,何富桂,20220302,2,1
44,倪诗瑄,20220311,3,1
...,...,...,...,...
15,黄金燕,20220316,2,1
48,齐金钢(齐金刚),20220223,1,1
49,齐金钢(齐金刚),20220225,4,1
50,齐金钢(齐金刚),20220228,2,1


In [9]:
# 去掉差的数据
df.drop(df.loc[df['标签']=='-'].index, inplace=True)
df.drop(df.loc[df['标签'].isnull()].index, inplace=True)
df=df.reset_index()
df=df.drop(['index'],axis=1)
df

,姓名,治疗时间,刺激方案,标签
0,何富桂,20220221,1,1
1,何富桂,20220225,4,1
2,何富桂,20220228,3,1
3,何富桂,20220302,2,1
4,倪诗瑄,20220311,3,1
...,...,...,...,...
91,黄金燕,20220316,2,1
92,齐金钢(齐金刚),20220223,1,1
93,齐金钢(齐金刚),20220225,4,1
94,齐金钢(齐金刚),20220228,2,1


In [10]:
# 将标签列数据类型转换为int
df["标签"] = df["标签"].astype(int)  
df

,姓名,治疗时间,刺激方案,标签
0,何富桂,20220221,1,1
1,何富桂,20220225,4,1
2,何富桂,20220228,3,1
3,何富桂,20220302,2,1
4,倪诗瑄,20220311,3,1
...,...,...,...,...
91,黄金燕,20220316,2,1
92,齐金钢(齐金刚),20220223,1,1
93,齐金钢(齐金刚),20220225,4,1
94,齐金钢(齐金刚),20220228,2,1


In [11]:
# rest_path = r"../patient_data/静息脑电（第一批次）/"
therapy_path=[r"../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考/"]

# rest_subdirectory = sorted(glob.glob(rest_path+"*"))
# therapy_subdirectory = sorted(glob.glob(therapy_path+"*"))
therapy_file_path=[]
for path in therapy_path:
    therapy_file_path = therapy_file_path+sorted(glob.glob(path+"*"))
#这里必须整体重新排序
therapy_file_path=sorted(therapy_file_path,key=lambda x:x.replace('\\','/').split('/')[-1].split('-')[1])
len(therapy_file_path)

96

In [12]:
therapy_file_path

['../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220221.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220225.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220228.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220302.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220311.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220314.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220316.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220318.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220309.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220311.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220314.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220316.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220225.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220228.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220302.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-2022030

In [13]:
path_parameter_dict={}
path_result_dict={}

In [14]:
df

,姓名,治疗时间,刺激方案,标签
0,何富桂,20220221,1,1
1,何富桂,20220225,4,1
2,何富桂,20220228,3,1
3,何富桂,20220302,2,1
4,倪诗瑄,20220311,3,1
...,...,...,...,...
91,黄金燕,20220316,2,1
92,齐金钢(齐金刚),20220223,1,1
93,齐金钢(齐金刚),20220225,4,1
94,齐金钢(齐金刚),20220228,2,1


In [15]:
therapy_file_path

['../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220221.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220225.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220228.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220302.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220311.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220314.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220316.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220318.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220309.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220311.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220314.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220316.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220225.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220228.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220302.edf',
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-2022030

In [16]:
#构建一个 {路径:ECT参数} 的字典，方便打乱顺序后，仍能保持一一对应
for i in range(len(therapy_file_path)):
    path_parameter_dict[therapy_file_path[i]]=df.iloc[i,2]
    path_result_dict[therapy_file_path[i]]=df.iloc[i,3]

In [17]:
path_parameter_dict

{'../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220221.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220225.edf': 4,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220228.edf': 3,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220302.edf': 2,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220311.edf': 3,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220314.edf': 4,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220316.edf': 2,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220318.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220309.edf': 4,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220311.edf': 2,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220314.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220316.edf': 3,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220225.edf': 4,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220228.edf': 2,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220302.edf': 3,
 '../patie

In [18]:
path_result_dict

{'../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220221.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220225.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220228.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220302.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220311.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220314.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220316.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220318.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220309.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220311.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220314.edf': 0,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220316.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220225.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220228.edf': 1,
 '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220302.edf': 1,
 '../patie

In [19]:
#观察文件名，发现所有文件名均为“-人名-”，所以取人名前后对照，如果不同则表示是不同的人
#思路：首先将所有人名取出，然后随机打乱顺序，前20个人作为训练数据，后4个人作为测试数据
therapy_file_path[0].replace('\\','/').split('/')[-1].split('-')[1]

'何富桂'

In [20]:
#获得名单
def getNameList(file_path):
    name_list=[]
    name_path_dict={}
    temp=[]
    cur_name='anonymous'
    for i in file_path:
        next_name=i.replace('\\','/').split('/')[-1].split('-')[1]
        if cur_name!=next_name:
            name_path_dict[cur_name]=temp
            cur_name=next_name
            name_list.append(cur_name)
            temp=[]             
        temp.append(i)      
    #也要把最后一个人写入词典
    name_path_dict[cur_name]=temp
    return name_list,name_path_dict

In [21]:
name_list,name_path_dict=getNameList(therapy_file_path)

In [22]:
name_path_dict

{'anonymous': [],
 '何富桂': ['../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220221.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220225.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220228.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-何富桂-20220302.edf'],
 '倪诗瑄': ['../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220311.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220314.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220316.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-倪诗瑄-20220318.edf'],
 '刘世清': ['../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220309.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220311.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220314.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘世清-20220316.edf'],
 '刘家瑞': ['../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220225.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-20220228.edf',
  '../patient_data/治疗脑电-导出/第一批次治疗脑电/平均参考\\AV-刘家瑞-202203

In [23]:
#打乱名单，体现随机性
np.random.seed(42)
np.random.shuffle(name_list)
len(name_list)

24

In [24]:
#没必要划分训练集测试集
if category=='train':
    train_patient_num=24
else:
    train_patient_num=0

total_patient_num=len(name_list)
train_therapy_file_path=[]
test_therapy_file_path=[]
for i in range(train_patient_num):
    for j in name_path_dict[name_list[i]]:
        train_therapy_file_path.append(j)
for i in range(train_patient_num,total_patient_num):
    for j in name_path_dict[name_list[i]]:
        test_therapy_file_path.append(j)

In [25]:
#这里必须要排序，注意排序的key
train_therapy_file_path.sort(key=lambda x:x.replace('\\','/').split('/')[-1].split('-')[1])
test_therapy_file_path.sort(key=lambda x:x.replace('\\','/').split('/')[-1].split('-')[1])

In [26]:
len(train_therapy_file_path)

0

In [27]:
df

,姓名,治疗时间,刺激方案,标签
0,何富桂,20220221,1,1
1,何富桂,20220225,4,1
2,何富桂,20220228,3,1
3,何富桂,20220302,2,1
4,倪诗瑄,20220311,3,1
...,...,...,...,...
91,黄金燕,20220316,2,1
92,齐金钢(齐金刚),20220223,1,1
93,齐金钢(齐金刚),20220225,4,1
94,齐金钢(齐金刚),20220228,2,1


In [28]:
#统计未发作和发作的样本数
trigger_num=0
non_trigger_num=0
for i in range(df.shape[0]):
    # 找不到未发作，即发作
    if df.iloc[i, 3]==1:
        trigger_num+=1
    else:
        non_trigger_num+=1
print(trigger_num/non_trigger_num)

multiple=trigger_num/non_trigger_num

4.333333333333333


In [29]:
multiple = round(multiple)
multiple

4

In [30]:
non_trigger_num

18

In [31]:
trigger_num+non_trigger_num*multiple

150

In [32]:
#抖动
def jitter(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            #抖动因子
            jitter_factor=np.random.randint(1,11)
            #抖动方向
            jitter_direction=np.random.randint(1,3)
            #变化在 *0.9 至 *1.1内
            if jitter_direction==1:
                data[i][j]= data[i][j]*(1+jitter_factor/100)
            else:
                data[i][j]= data[i][j]*(1-jitter_factor/100)

In [33]:
#翻转
def flip(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]=-data[i][j]

In [34]:
#随机通道翻转
def random_flip(data):
    flip_channel=np.random.randint(0,15)
    for i in range (data.shape[1]):
        data[flip_channel][i]=-data[flip_channel][i]

In [35]:
#扩大/压缩
def scale_up(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]= data[i][j]*1.1
def scale_down(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]= data[i][j]*0.9

In [36]:
# #排列
# def permutation(data,p_strategy):
#     for i in range(0,16,2):
#         #把2s片段分为4组,假设序号为1,2,3,4
#         p1_start=int(i*1024)
#         p1_end=int((i+0.5)*1024)
#         p2_start=int((i+0.5)*1024)
#         p2_end=int((i+1)*1024)
#         p3_start=int((i+1)*1024)
#         p3_end=int((i+1.5)*1024)
#         p4_start=int((i+1.5)*1024)
#         p4_end=int((i+2)*1024)
#         for j in range(15):
#             #排列方案为1时， 1,2,3,4-->2,3,4,1
#             if p_strategy==1: 
#                 data_temp=[]
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data=np.array(data)
#                 data[j,p1_start:p4_end]=data_temp
#             #排列方案为2时， 1,2,3,4-->3,4,1,2
#             elif p_strategy==2:
#                 data_temp=[]
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data[j,p1_start:p4_end]=data_temp
#             #排列方案为3时， 1,2,3,4-->4,1,2,3
#             elif p_strategy==3:
#                 data_temp=[]
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data[j,p1_start:p4_end]=data_temp

In [37]:
#检测EEG文件是否有flag标记,返回true/false
def judgeflag(file_path,flag):
    raw = read_raw_edf(file_path,preload=False)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    if flag in event_dict:
        return True
    else:
        return False

In [38]:
# raw=read_raw_edf(train_therapy_file_path[0],preload=False)

In [39]:
# events_from_annot, event_dict = mne.events_from_annotations(raw)

In [40]:
# event_dict

In [41]:
# events_from_annot

In [42]:
# data,times=raw[:,0:1024*16]
# raw2=mne.io.RawArray(data,raw.info)

In [43]:
raw=read_raw_edf(test_therapy_file_path[0],preload=False)

Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第一批次治疗脑电\平均参考\AV-何富桂-20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [44]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: EEG Fp1-AV, EEG Fp2-AV, EEG F3-AV, EEG F4-AV, EEG C3-AV, EEG ...
 chs: 19 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 512.0 Hz
 meas_date: 2022-02-21 16:02:04 UTC
 nchan: 19
 projs: []
 sfreq: 1024.0 Hz
>

In [45]:
# 数据采样率1024Hz,文件路径；采集时间；是否为测试集标志;数据增强策略;未发作的第几次随机通道翻转
def get_se_epochs(path,select_time,is_test,DA_strategy,random_flip_no):
    #选择15个通道
    chs1 = ['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV',
       'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV','EEG Cz-AV']
#     chs2 =['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1','EEG F4-A2','EEG C3-A1','EEG C4-A2','EEG P3-A1','EEG P4-A2','EEG O1-A1',
#            'EEG O2-A2','EEG T3-A1','EEG T4-A2','EEG T5-A1','EEG T6-A2','EEG Cz-AAV']
    # select_time 采集的时间
    raw=read_raw_edf(path,preload=False)
    print(raw.info.ch_names)
    #第一批数据采样为1024Hz，第二批变成了512Hz
#     if chs1[0] in raw.info.ch_names:
#         chs=chs1  
#         freq=1024
#     else:
#         chs=chs2
#         freq=512
    chs=chs1
    freq=1024
    raw=raw.pick_channels(chs)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    # 找到SE，麻醉标记,save函数的参数单位是s，所以要除以1024
    start_time=events_from_annot[event_dict['SE']-2][0]
    end_time=start_time+select_time*freq
    print(start_time,end_time)
    #选择所有的15通道
    data,times = raw[:,start_time:end_time]
    print(times)
    #不做变化
    if (DA_strategy==0): 
        path_name_suffix='(original)'
    #抖动
    elif (DA_strategy==1):
        jitter(data)
        path_name_suffix='(jitter)'
    #翻转
    elif (DA_strategy==2):
        flip(data)
        path_name_suffix='(flip)'
    #扩大
    elif (DA_strategy==3):
        scale_up(data)
        path_name_suffix='(scale_up)'  
    #缩小
    elif (DA_strategy==4):
        scale_down(data)
        path_name_suffix='(scale_down)'  
    #排列1
    elif (DA_strategy==5):
        permutation(data,1)
        path_name_suffix='(permutation_1)'  
    #排列2
    elif (DA_strategy==6):
        permutation(data,2)
        path_name_suffix='(permutation_2)'  
    #排列3
    elif (DA_strategy==7):
        permutation(data,3)
        path_name_suffix='(permutation_3)'  
    #随机通道翻转
    elif (DA_strategy==8):
        random_flip(data)
        path_name_suffix='random_flip_'+str(random_flip_no)
    raw_temp=mne.io.RawArray(data,raw.info)
    path_name=path.split('\\')[1:]
    path_name="".join(path_name)
    path_name=path_name.split('.')[0]
    #在这里将训练集和测试集分开,需要新建SE_train和SE_test两个文件夹
    if (is_test==False):
        path_name=r"../SE_testdata/" + path_name +path_name_suffix+".fif"
    else:   
        path_name=r"../SE_testdata/" + path_name + ".fif"     
    path_name=path_name.replace('/','\\')
    print(path_name)
    raw_temp.save(path_name,overwrite=True)

In [46]:
file_without_se = []
df_parameter_train=pd.DataFrame(columns=['parameter','result'])
df_parameter_test=pd.DataFrame(columns=['parameter','result'])

In [47]:
train_therapy_file_path

[]

In [48]:
#训练集SE切片长度
train_SE_len=16
#测试集SE切片长度
test_SE_len=16
#选择offset是多少的倍数
# offset_multiple=2

In [49]:
#生成测试数据
for file_path in train_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #初始化random_flip_no，random_flip_no的作用是方便命名未发作文件，防止发生同名文件覆盖
        random_flip_no=0
        #如果发作，则只截取train_SE_len长度
        #为False时表示当前数据送入训练集
        #通过8种策略数据增强
        for DA_strategy in range(5):
            get_se_epochs(file_path,train_SE_len,False,DA_strategy,0)
            #添加参数到df_parameter_train的末尾
            df_parameter_train.loc[len(df_parameter_train.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
            #如果未发作，则随机通道翻转增加未发作样本数量
            if path_result_dict[file_path]==0:
                for j in range(1,multiple):
                    #修改random_flip_no
                    random_flip_no+=1
                    #DA_strategy固定为8，即通过随机翻转达到0-1平衡
                    get_se_epochs(file_path,train_SE_len,False,8,random_flip_no) 
                    df_parameter_train.loc[len(df_parameter_train.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
    else:
        file_without_se.append(file_path)
#生成SE_test,不考虑对未发作数据的重叠采样
for file_path in test_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #取test_SE_len片段
        #DA_strategy=0,即不做变化
        get_se_epochs(file_path,test_SE_len,True,0,0) 
        df_parameter_test.loc[len(df_parameter_test.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
    else:
        file_without_se.append(file_path)

Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第一批次治疗脑电\平均参考\AV-何富桂-20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第一批次治疗脑电\平均参考\AV-何富桂-20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
8919 25303
[ 8.70996094  8.7109375   8.71191406 ... 24.70703125 24.70800781
 24.70898438]
Creating RawArray with float64 data, n_channels=15, n_times=16384
    Range : 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-何富桂-20220221.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-何富桂-20220225.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filen

EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
132169 148553
[129.07128906 129.07226562 129.07324219 ... 145.06835938 145.06933594
 145.0703125 ]
Creating RawArray with float64 data, n_channels=15, n_times=16384
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
..\SE_testdata\AV-倪诗瑄-20220316.fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-倪诗瑄-20220316.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-倪诗瑄-20220316.fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第一批次治疗脑电\平均参考\AV-倪诗瑄-20220318.edf...
EDF file detected
Setting cha

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-倪诗瑄-20220316.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-倪诗瑄-20220318.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filen

Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
46143 62527
[45.06152344 45.0625     45.06347656 ... 61.05859375 61.05957031
 61.06054688]
Creating RawArray with float64 data, n_channels=15, n_times=16384
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
..\SE_testdata\AV-刘世清-20220316.fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-刘世清-20220316.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-刘世清-20220316.fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第一批次治疗脑电\平均参考\AV-刘家瑞-20220225.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第一批次治疗脑电\平均参考\AV-刘家瑞-20220225.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure

...

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-雷震-20220328.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-黄盈雪-20220314.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filena

Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
75866 92250
[74.08789062 74.08886719 74.08984375 ... 90.08496094 90.0859375
 90.08691406]
Creating RawArray with float64 data, n_channels=15, n_times=16384
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
..\SE_testdata\AV-黄金燕-20220311.fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-黄金燕-20220311.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-黄金燕-20220311.fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第一批次治疗脑电\平均参考\AV-黄金燕-20220314.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第一批次治疗脑电\平均参考\AV-黄金燕-20220314.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure.

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-黄金燕-20220311.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-黄金燕-20220314.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filen

Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
150366 166750
[146.84179688 146.84277344 146.84375    ... 162.83886719 162.83984375
 162.84082031]
Creating RawArray with float64 data, n_channels=15, n_times=16384
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
..\SE_testdata\AV-齐金钢-20220302.fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-齐金钢-20220302.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-齐金钢-20220302.fif
[done]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\3871462243.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata\AV-齐金钢-20220302.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)


In [50]:
file_without_se

[]

In [51]:
df_parameter_test

,parameter,result
0,1,1
1,4,1
2,3,1
3,2,1
4,3,1
...,...,...
91,2,1
92,1,1
93,4,1
94,2,1


In [52]:
output_path=("../SE_parameter/testmodel_matched_stimulate_parameter.xlsx")

In [53]:
if category=='train':
    df_parameter_train.to_excel(output_path,sheet_name="Sheet1")
else:
    df_parameter_test.to_excel(output_path,sheet_name="Sheet1")

In [54]:
# 引入工具包
from mne.io import concatenate_raws,read_raw_fif
import matplotlib.pyplot as plt
import mne
import tensorflow as tf
import numpy as np
from numpy.lib.utils import safe_eval
from keras.models import load_model
import keras.backend as K
import glob
import scipy.signal
import scipy.io as scio
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [55]:
def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)
# % 模型准备 %
model_path = r'../others/Gao_Novel_CNN_RNN16_2561630689427.h5'
model = load_model(model_path, custom_objects={'pearson_r': pearson_r})

In [56]:
def SE_process(file_path,save_path,select_time):
    raw = read_raw_fif(file_path)
    #改成了15通道
    data,times=raw[:,:]
    
    samps = int(select_time * 256)
    
    # 重采样为256Hz
    data_256 = []
    for i in range(data.shape[0]):
            data_temp = scipy.signal.resample(data[i],samps)
            data_256.append(data_temp)
    # 重新转换为np.array
    data_256 = np.array(data_256)                                    
    print("data_256.shape",data_256.shape)
    
    # 分成 epoches，256*2的形状
    data_256_samples = []
    # 对每个通道
    for i in range(data_256.shape[0]):
        # 进行片段切片
        channel = data_256[i];
        # samps = 2*256 两秒，每秒256个采样点
        data_channels = []
        for j in range(0,channel.shape[0],2*256):
            # print("第" + str(j//samps) + "个片段")
            data_temp = channel[j:j+2*256]
            data_channels.append(data_temp)
        # 每个通道整合起来
        data_256_samples.append(data_channels)

    # 转回np.array
    data_256_samples = np.array(data_256_samples)
    print("data_256_samples.shape",data_256_samples.shape)

    # 进行标准化处理,sta表示Standardization
    data_sta = []
    # 对每个通道
    for i in range(data_256_samples.shape[0]):
        # np.mean和np.std计算均值和标准差
        tmp_mean = np.mean(data_256_samples[i])
        tmp_std = np.std(data_256_samples[i])
        data_temp_std = (data_256_samples[i] - tmp_mean) / tmp_std
        data_sta.append(data_temp_std)
    # 转回np.array
    data_sta = np.array(data_sta)
    print("data_sta.shape",data_sta.shape) 

    # 整理成神经网络训练所需的形状，这里把数据展平成一维，所有通道合并，之后再还原
    data_x = data_sta.reshape(-1,512,1)
    print("data_x.shape:",data_x.shape)
    # 模型进行预测
    y_pred = model.predict(data_x, verbose=0)
    # 先把通道分开
    # data_256_samples.shape[1] = 30 即降采样后的片段数
    
    pred_channel = []

    for i in range(0,y_pred.shape[0],data_256_samples.shape[1]):
        channel_temp = y_pred[i:i+data_256_samples.shape[1]]
        channel_temp = channel_temp.flatten()
        pred_channel.append(channel_temp)

    pred_channel = np.array(pred_channel)
    print("pred_channel.shape",pred_channel.shape)
    
    #改成了15通道
    info = mne.create_info(ch_names=raw.ch_names[:15],ch_types=["eeg"]*15,sfreq=256)
    """
    利用mne.io.RawArray类创建Raw对象
    """
    custom_raw = mne.io.RawArray(data=pred_channel, info=info)
    custom_raw.save(save_path,overwrite=True)
    
    
    return True

In [57]:
train_directory = glob.glob("../SE_testdata/"+"*")

In [58]:
train_SE_len=16

In [59]:
for file_path in train_directory:
    file_name = file_path.split("\\")[-1]
    save_path = "../SE_testdata_processed/" + "".join(file_name)
    print(save_path)
    #训练集为16s
    SE_process(file_path,save_path,train_SE_len)

../SE_testdata_processed/AV-何富桂-20220221.fif
Opening raw data file ../SE_testdata\AV-何富桂-20220221.fif...
Isotrak not found
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
data_256.shape (15, 4096)
data_256_samples.shape (15, 8, 512)
data_sta.shape (15, 8, 512)
data_x.shape: (120, 512, 1)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:2: RuntimeWarning: This filename (../SE_testdata\AV-何富桂-20220221.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)


pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220221.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220221.fif
[done]
../SE_testdata_processed/AV-何富桂-20220225.fif
Opening raw data file ../SE_testdata\AV-何富桂-20220225.fif...
Isotrak not found
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
data_256.shape (15, 4096)
data_256_samples.shape (15, 8, 512)
data_sta.shape (15, 8, 512)
data_x.shape: (120, 512, 1)
pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220225.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220225.fif
[done]
../SE_te

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:73: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220221.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  custom_raw.save(save_path,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:2: RuntimeWarning: This filename (../SE_testdata\AV-何富桂-20220225.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:73: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work

pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220228.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220228.fif
[done]
../SE_testdata_processed/AV-何富桂-20220302.fif
Opening raw data file ../SE_testdata\AV-何富桂-20220302.fif...
Isotrak not found
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
data_256.shape (15, 4096)
data_256_samples.shape (15, 8, 512)
data_sta.shape (15, 8, 512)
data_x.shape: (120, 512, 1)
pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220302.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-何富桂-20220302.fif
[done]
../SE_te

...

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:73: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-黄金燕-20220314.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  custom_raw.save(save_path,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:2: RuntimeWarning: This filename (../SE_testdata\AV-黄金燕-20220316.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:73: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work

    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220223.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220223.fif
[done]
../SE_testdata_processed/AV-齐金钢-20220225.fif
Opening raw data file ../SE_testdata\AV-齐金钢-20220225.fif...
Isotrak not found
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
data_256.shape (15, 4096)
data_256_samples.shape (15, 8, 512)
data_sta.shape (15, 8, 512)
data_x.shape: (120, 512, 1)
pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220225.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220225.fif
[done]
../SE_testdata_processed/AV-齐金钢-20220228.fif
Opening raw data file ../SE_testdata\AV-齐金钢-20220228.fif..

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:73: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220223.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  custom_raw.save(save_path,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:2: RuntimeWarning: This filename (../SE_testdata\AV-齐金钢-20220225.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:73: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work

pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220228.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220228.fif
[done]
../SE_testdata_processed/AV-齐金钢-20220302.fif
Opening raw data file ../SE_testdata\AV-齐金钢-20220302.fif...
Isotrak not found
    Range : 0 ... 16383 =      0.000 ...    15.999 secs
Ready.
data_256.shape (15, 4096)
data_256_samples.shape (15, 8, 512)
data_sta.shape (15, 8, 512)
data_x.shape: (120, 512, 1)
pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220302.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220302.fif
[done]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:73: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_testdata_processed\AV-齐金钢-20220228.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  custom_raw.save(save_path,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:2: RuntimeWarning: This filename (../SE_testdata\AV-齐金钢-20220302.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_20340\2822037444.py:73: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work

In [60]:
print("finish")

finish
